# Notebook for End to End testing in the MID PSI

###### Last updated 01/08/24

This notebook Uses Dish LMC, SPFRx, TMC, SDP, CBF and CSP.LMC to run an end-to-end scan. It also provides the option to run multiple offset scan to enable calibration, following the [Five Point Calibration Scan Controls notebook](https://gitlab.com/ska-telescope/ska-jupyter-scripting/-/blob/main/notebooks/observing/MID_five_point_calibration_scan_controls.ipynb?ref_type=heads).

To use Dish LMC/SPFRx, ensure the namespace to use with this project is started with `DISH_LMC_ENABLED` set to `true`. Otherwise, the default setting of `false` can be used. 

If using SPFRx, only Talon1 can be used (as it is what the spfrx is connected to) and the rxpu must be signed out.

### For using BITE

BITE Functionality has been spun out to the [bite_generation notebook](bite_generation.ipynb), which should be referred to when needed. These steps are labeled as BITE STEP and are:
- Setting variables
- Loading Config Data
- Generating BITE Data
- Starting LSTV Replay
- Stopping LSTV Replay

## 1 Setup

### 1.1 Environment Setup

Start by importing all the libraries needed for this notebook

In [ ]:
import sys

sys.path.append("../../src")

import json
import os
import time
from time import sleep

from IPython.display import clear_output
from tango import Database, DevFailed, DeviceProxy

import notebook_tools.generate_fsp as generate_fsp
import notebook_tools.wait_for_tango as wait_for_tango

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [ ]:
!kubectl get ns | grep ska-mid-psi

Now, use this to set the namespace the notebook will use.

In [ ]:
psi_namespace = "ci-ska-mid-psi-1413983681-alexschell"  # Namespace to be used
fsp_count = 1  # Number of FSPs to run with (Should be less than or equal to number of boards)
using_spfrx = True  # Set to true if using spfrx, otherwise if using BITE can be set to False
target_boards = [
    1
]  # Talon board(s) to use for the notebook. If using SPFRx, only Talon1 can be used.

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<a id='setting bite vars'></a>
<div class="alert">
    <h3>(BITE STEP) Set Variables in the BITE Notebook</h3>
    <br>
    If running the BITE notebook, at this point ensure the variables for namespace and test_id are properly set.
</div>
</body>

Now, load in the other variables this notebook will use, along with config files to pass in. These vars should not need to be changed in most use cases.

In [ ]:
TANGO_HOST = f"databaseds-tango-base.{psi_namespace}.svc.cluster.local:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/psi/tmc"

# TMC config files
ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{TMC_CONFIGS}/release_resources.json"

# For running offset scans
SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

# CBF dish files
CBF_CONFIGS = f"{DATA_DIR}/psi/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"
HW_CONFIG_FOLDER = os.path.join(CBF_CONFIGS, "hw_config")
INIT_SYS_PARAM_FILE = os.path.join(CBF_CONFIGS, "sys_params/initial_system_param.json")

# Select HW file based on boards selected
if any(i > 4 for i in target_boards):
    print("Using swap for higher number talons")
    hw_config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards = list(map(lambda x: x - 4, target_boards))
else:
    print("Using standard HW config")
    hw_config = "hw_config_psi.yaml"
HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, hw_config)

# Check files can be reached.
files = [
    HW_CONFIG_FILE,
    INIT_SYS_PARAM_FILE,
    DISH_CONFIG_FILE,
    ASSIGN_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    RELEASE_RESOURCES_FILE,
]

# Slim config files setup
# Load in nothing if using 1 board as not needed, load in 4vcc 1fsp if needed
print("Checking SLIM configs:")
if len(target_boards) == 1:
    slim_fs_config = ""
    slim_vis_config = ""
else:
    slim_fs_config = "fs_slim_4vcc_1fsp.yaml"  # update if necessary
    slim_vis_config = ""  # update if necessary

SLIM_CONFIGS = os.path.join(CBF_CONFIGS, "slim_config")
SLIM_FS_CONFIG_FILE = os.path.join(SLIM_CONFIGS, slim_fs_config)
SLIM_VIS_CONFIG_FILE = os.path.join(SLIM_CONFIGS, slim_vis_config)

if slim_fs_config != "":
    print("    Loading custom SLIM fs config")
    !kubectl cp $SLIM_FS_CONFIG_FILE $PSI_NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml
    files.append(SLIM_FS_CONFIG_FILE)
else:
    print("    SLIM fs will use default config")

if slim_vis_config != "":
    print("    Loading custom SLIM vis config")
    !kubectl cp $SLIM_VIS_CONFIG_FILE $PSI_NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml
    files.append(SLIM_VIS_CONFIG_FILE)
else:
    print("    SLIM vis will use default config")

# Ensure the files exist
print("Checking to ensure files exist:")
for file in files:
    if os.path.isfile(file):
        print(f"    {file} exists: ✔️")
    else:
        print(f"    {file} does not exist ❌")

# Map the talon boards to receptor IDs
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]
RECEPTORS = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards))
# Sanity check to ensure that Receptors = talons match
if len(RECEPTORS) == len(target_boards):
    print("Receptors match number of talons")
else:
    print("Receptor/talon mismatch!")
# Sanity check to ensure the number of FSPs <= number of boards
if fsp_count <= len(target_boards):
    print("FSP count is correct.")
else:
    print(f"Error: Incorrect number of FSPs, should be less than or equal to {len(target_boards)}")
print(f"Targeting receptors: {RECEPTORS}")

If using multiple boards, the SLIM configuration will have to be uploaded:

In [ ]:
if len(target_boards) > 1:
    if slim_fs_config != "":
        print("Loading custom SLIM fs config")
        !kubectl cp $SLIM_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/slim/fs_slim_config.yaml
    else:
        print("SLIM fs will use defaults for this test.")
    if slim_vis_config != "":
        print("Loading custom SLIM vis config")
        !kubectl cp $SLIM_CONFIG_FILE $ns/ds-cbfcontroller-controller-0:/app/mnt/slim/vis_slim_config.yaml
    else:
        print("SLIM vis will use default config")
else:
    print("SLIM configuration not required as only using one board...")

Along with these files, the hardware config can also be loaded into the pod via kubectl:

In [ ]:
!kubectl cp $HW_CONFIG_FILE $psi_namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

With the namespace set, the useful front ends to monitor the behaviour of the system can be accessed using the following URLs.

In [ ]:
print("For showing the signal output:")
print(f"https://142.73.34.170/{psi_namespace}/signal/display/")
print("For working with and checking all the TANGO devices currently in the namespace:")
print(f"https://142.73.34.170/{psi_namespace}/taranta/devices")
print("For monitoring the status of the tango devices:")
print(
    f"https://142.73.34.170/{psi_namespace}/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run"
)
print("For using the EDA configurator (Only if SKA_TANGO_ARCHIVER=true for the namespace):")
print(f"https://142.73.34.170/{psi_namespace}/configurator/")

### 1.3 Setup Device Proxies

First, to interact with the devices used by this notebook, TANGO device proxies must be set up to connect to and control them.

In [ ]:
# TMC Proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP Proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF Proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Dish Leaf Proxies
dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")

# SDP Proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Leaf Node Proxies
csp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
sdp_subarray_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/sdp_subarray01")
csp_master_leaf_node = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

# Deployer Proxy
deployer = DeviceProxy("mid_csp_cbf/ec/deployer")

# print the states of each
devices = [
    tmc_central_node,
    tmc_csp_master,
    tmc_csp_subarray,
    tmc_subarray,
    csp_control,
    csp_subarray,
    cbf_controller,
    cbf_subarray,
    dish_leaf_node_ska001,
    sdp_subarray,
    csp_subarray_leaf_node,
    sdp_subarray_leaf_node,
    csp_master_leaf_node,
    deployer,
]

for device in devices:
    padding = "-" * (40 - len(device.dev_name()))
    print(f"{device.dev_name()}'s state {padding}> {device.state()}")

### 1.4 MCS Deployer Setup and Download Artifacts

To use the deployer, set the dish ID that will be deployed to.

In [ ]:
db = Database()
deployer.targetTalons = target_boards
print("Deployer will target the following talons:", deployer.targetTalons)
deployer.generate_config_jsons()

Now the actual download step can be run, this will take some time.

In [ ]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Once the downloaded, the TANGO device database can be configured with the new downloads

In [ ]:
deployer.configure_db()

If desired, the devices can be checked to ensure they have been downloaded.

In [ ]:
print(*db.get_device_exported("*").value_string, sep="\n")

## 2 Set Up Devices

### 2.1 Setting up the CSP/CBF

With the connection established to the devices, set the admin and simulation mode to both be 0. This will allow the running of commands and ensure real hardware is being used.

In [ ]:
# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0
sleep(1)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

In [ ]:
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

### 2.2 Load the Dish Vcc Config / Init Sys Params

Next, load in the dish config file to the central node:

**Note: This may need to be run multiple times**

In [ ]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

# Reach out to grab the tagged Telescope model with K-value of 1
# See: https://gitlab.com/ska-telescope/ska-telmodel-data/-/tree/0.1.0-rc-mid-itf/tmdata/instrument/ska1_mid_itf
dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

# Actually load in the dish config
print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))
# Wait for dishvcc to be loaded...
wait_seconds = 0
while not tmc_central_node.isDishVccConfigSet:
    clear_output(wait=True)
    print(f"Waiting for DishVCC to be set, {wait_seconds} seconds elapsed...")
    sleep(2)
    wait_seconds += 2

clear_output(wait=True)
print("DishVCC has been set!")
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Load Config Data</h3>
    <br>
    Now, if required, the BITE configuration data can be loaded in using the BITE notebook.
</div>
</body>

### 2.3 Turn the Telescope On

Now, turn the telescope itself on:

In [ ]:
print("Running the TelescopeOn command")
tmc_central_node.set_timeout_millis(100000)
tmc_central_node.TelescopeOn()

startup_time = 0
alert_msg = ""
while int(tmc_central_node.telescopeState) != 0:
    print(f"\r Telescope is starting up, {startup_time} seconds elapsed. {alert_msg}", end="")
    sleep(5)
    startup_time += 5
    if startup_time > 120:
        alert_msg = "Startup is taking longer than expected, try running LRU power off scripts."
print(f"\n Telescope has started after {startup_time} seconds.")

#### 2.3.1 Remedying Telescope on 

In some cases, the steps for powering on will not complete, due to DDR calibration failing, causing the HPS to error out. This can be checked by monitoring the CBF controller and logconsumer to see if a HPS error has occurred. If monitoring central node while running, these errors will likely surface as timeout errors when running the above step
Check if these error messages occur: 

- hpsmaster (via ds-talonlogconsumer device): 
   - `DsHpsMaster::configure: Timeout waiting for Talon Status`

- ds-cbfcontroller-controller device: 
   - `Configure command for talondx-.../hpsmaster/hps-2 device failed with error code 4`
   - `Failed to configure Talon boards`
   - `Exiting command OnCommand with return_code ResultCode.FAILED, message: 'Failed to configure Talon boards'.`

If this is the case, the LRU will have to be powered down, then powered on using the scripts available through talon_power_lru.sh. **Note that this will require that both boards on each LRU (like 1 and 2) will be shut off**. See [CIP-2344](https://jira.skatelescope.org/browse/CIP-2344) for more details.

If it is required to reset the LRU, on a dev machine run the following steps:

***

Once the On command has been run successfully, check the states of all the devices and ensure they are on (and 0 for the central node).

In [ ]:
print("Verifying the states:")
print(f"  TMC Central Node State: {int(tmc_central_node.TelescopeState)}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Generate BITE Data</h3>
    <br>
    With the telescope on, the section of the BITE generation notebook that handles the actual generation of BITE data can be run.
</div>
</body>

### 2.4 Assign Resources to the Telescope

Start assigning resources by setting up the JSON file as needed:

In [ ]:
with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS

print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

Now, the command to actually assign the resources can be run, and the TMC should go to idle (2)

In [ ]:
tmc_subarray.AssignResources(json.dumps(assign_resources_json))
wait_for_tango.wait_for_state(tmc_subarray, 2)

If desired to simulate the whole setup of a real telescope, the slew can be set via the dish manager (if using BITE, this can be skipped):

In [ ]:
if using_spfrx:
    dish_manager = DeviceProxy("mid-dish/dish-manager/ska001")
    dish_manager.slew([181.0, 31.0])

    while dish_manager.achievedpointing[1] != 181.0 or dish_manager.achievedpointing[2] != 31.0:
        time.sleep(1)

    print("Done slewing!")
else:
    print("Using BITE, slew not required.")

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Start LSTV Replay</h3>
    <br>
    If using BITE for data stream generation, the respective step in the BITE notebook can be run at this point.
</div>
</body>

## 3 Running the Scan(s)

With everything set up, the scans can now be run:

### 3.1 Configure Scan

Before running the scan configuration, ensure that the SDP vis pod has spun up:

In [ ]:
!kubectl -n $psi_namespace-sdp get pods | grep vis-receive

Now, append the configure scan JSON file as needed before uploading it.

In [ ]:
with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

# Create and Append FSPs to scan config file
# note that channel offset will change with ADR-99
configure_scan_json["csp"]["cbf"]["fsp"] = []
fsp_list = generate_fsp.generate_fsp_list(fsp_count, target_boards)
configure_scan_json["csp"]["cbf"]["fsp"] = fsp_list

# Assign the config ID
configure_scan_json["csp"]["common"][
    "config_id"
] = f"{len(target_boards)} receptor, band 1, {fsp_count} FSP, no options"

print("Appended configure scan file:")
print(json.dumps(configure_scan_json, indent=1))

Send the configuration, and wait for the TMC to go to ready:

In [ ]:
tmc_subarray.Configure(json.dumps(configure_scan_json))
wait_for_tango.wait_for_state(tmc_subarray, 4)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### 3.2 Running a Non-Offset Scan

Now, the scan itself can be run by sending the command to the TMC:

**Currently this method errors out with a validation error, use the following cell to get around this**

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning (5)")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json file contents: \n{scan_json}")

tmc_subarray.Scan(scan_json)
wait_for_tango.wait_for_state(cbf_subarray, 5)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

For the moment, run this instead to get around the configuration issue:

In [ ]:
with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node.sdpSubarrayObsState)

with open("../../data/psi/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node.cspSubarrayObsState)

Monitor the SDP vis pod and the signal display webpage to ensure the scan is underway.

### 3.3 Ending Initial Scan


After allowing the scan to run for a while, end it via the appropriate command:

**If using the workaround, send the endScan command via the TMC**

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready (4) state")

tmc_subarray.EndScan()
wait_for_tango.wait_for_state(tmc_subarray, 4)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

Otherwise send the EndScan via sdp/csp directly:

In [ ]:
sdp_subarray_leaf_node.EndScan()
csp_subarray_leaf_node.EndScan()

After the above cell, wait for the subarray to go to the Ready (4) state.

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### 3.4 Running Offset Scans for Multi-Point Calibration (Optional)

Now, loop through the offset scans, using the combos to set the scan json. For each, configure it by passing in JSON, then scan again, letting it run before stopping.

In [ ]:
scan_run = 1
for offset in SCAN_COMBOS:
    print(f"setting offset to: {offset}")
    # Configure for this offset scan
    partial_configure_json = {
        "interface": "https://schema.skao.int/ska-tmc-configure/2.2",
        "transaction_id": f"txn-....-0000{scan_run}",
        "scan_id": scan_run,
        "pointing": {"target": {"ca_offset_arcsec": offset[0], "ie_offset_arcsec": offset[1]}},
        "tmc": {"partial_configuration": True},
    }
    print("Partial Config to load:")
    print(json.dumps(partial_configure_json, indent=1))
    print(".......")
    sleep(10)
    csp_subarray_leaf_node.Configure(json.dumps(partial_configure_json))
    wait_for_tango.wait_for_state(cbf_subarray, 4)

    sdp_subarray_leaf_node.Configure(json.dumps(partial_configure_json))

    # Send the Scan command along with relevant JSON,incrementing the scan ID and transaction ID
    partial_scan_json = {
        "interface": "https://schema.skao.int/ska-tmc-scan/2.1",
        "transaction_id": f"txn-....-0000{scan_run}",
        "scan_id": scan_run,
    }
    print(json.dumps(partial_scan_json, indent=1))
    sdp_subarray_leaf_node.scan(json.dumps(partial_scan_json))
    sleep(10)
    csp_subarray_leaf_node.Scan(json.dumps(partial_scan_json))
    wait_for_tango.wait_for_state(cbf_subarray, 5)
    # let the scan run for a bit...
    sleep(30)
    # While the scan is running, refresh the signal page and monitor the vis pod logs to ensure the data is coming through.

    # End the scan
    sdp_subarray_leaf_node.EndScan()
    sleep(10)
    csp_subarray_leaf_node.EndScan()
    wait_for_tango.wait_for_state(cbf_subarray, 4)

    scan_run += 1
    print("============================")
print("Done offsets!")

## 4 Cleanup

Now that the scan(s) are complete, the namespace and devices can be shut down and cleaned up:

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Stopping LSTV Replay</h3>
    <br>
    Now with the scan(s) done, if using BITE, run the LSTV replay section of the BITE generation notebook.
</div>
</body>

Start by ending any running scans.

**For use when SKB-399 is resolved:**

In [ ]:
print("Running the End command: subarray obsstate should go to Idle (2) state")

tmc_subarray.End()
wait_for_tango.wait_for_state(tmc_subarray, 2)
print(f"\nTMC Subarray Observation State: {cbf_subarray.obsState}")

For now use:

In [ ]:
sdp_subarray_leaf_node.End()
csp_subarray_leaf_node.End()

And check that the subarray has gone to the IDLE (2) state.

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### 4.1 Release Resources

Once the scans are stopped and finished, clear the resources from the appropriate devices.

**For use when SKB-399 is resolved:**

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

tmc_subarray.ReleaseAllResources()
while tmc_subarray.obsState != 0:
    sleep(5)
    clear_output(wait=True)
    print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

For now use:

In [ ]:
sdp_subarray_leaf_node.ReleaseAllResources()
csp_subarray_leaf_node.ReleaseAllResources()

And ensure that the subarray obsstate goes to EMPTY (0). At this stage, the receptor IDs should also be empty.

In [ ]:
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node.cspSubarrayObsState}")

### 4.2 Turn the Telescope Off

Finally, send the Off command to the TMC

In [ ]:
print("Running the TelescopeOff command")
tmc_central_node.TelescopeOff()

while int(tmc_central_node.TelescopeState) != 1:
    sleep(5)
    clear_output(wait=True)
    print("These should all go to OFF")
    print(f"TMC Central Node State: {int(tmc_central_node.TelescopeState)}")
    print(f"CSP Control State: {csp_control.State()}")
    print(f"CBF Controller State: {cbf_controller.State()}")